# Lab 02
- Missing values
- Outliers
- Binning
- Transformations
- Categorical Encoding
- Scalling

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

In [ ]:
data.dtypes

## Missing values

In [ ]:
na_ratio_cols = data.isna().mean(axis=0)
na_ratio_cols

### Usuwanie kolumn

In [ ]:
data.shape

In [ ]:
# Usunięcie kolumn gdzie odsetek NA przekracza próg
data[data.columns[na_ratio_cols < 0.5]].shape

In [ ]:
# Usunięcie wszystkich kolumn gdzie występuje chociaż jedno NA
data.dropna(axis=1).shape

### Usuwanie rekordów

In [ ]:
na_ratio_rows = data.isna().mean(axis=1)
na_ratio_rows

In [ ]:
# Usunięcie wierszy gdzie odsetek NA przekracza próg
data.loc[na_ratio_rows < 0.1].shape

In [ ]:
# Usunięcie wszystkich wierszy gdzie występuje chociaż jedno NA
data.dropna(axis=0).shape

### Imputation

#### Fixed number imputation
- dlaczego jest lepszy niż usuwanie NA?
- kiedy ma sens?

In [ ]:
data_tmp = data.copy()

In [ ]:
# Wypełnienie NA w pojedynczych kolumnach
data_tmp['Age'] = data.Age.fillna(0)
data_tmp['Cabin'] = data.Cabin.fillna(-1)

# Wypełnienie NA we wszystkich kolumnach
data.fillna(-1).head(3)

#### Mean, median, ... imputation
- kiedy wybieramy średnią a kiedy medianę?

In [ ]:
data_tmp = data.copy()

In [ ]:
data.Age.mean(), data.Age.median()

In [ ]:
data_tmp['Age'] = data.Age.fillna(data.Age.mean(),inplace=True)
data_tmp['Age'] = data.Age.fillna(data.Age.median(),inplace=True)

#### Categorical imputation

In [ ]:
data.Cabin.value_counts()

In [ ]:
# Wykorzystanie najczęściej występującej kategorii
# - Czy to ma sens?
data.Cabin.fillna(data.Cabin.value_counts().idxmax())

In [ ]:
# Wypełnienie osobną kategorią - Other
data.Cabin.fillna('Other')

**Czy zaprezentowane sposoby uzupełnienia Nan są poprawne?**

## Outliers
- najlepiej wykrywa się wartości odstające na wykresie (np. boxplot)

In [ ]:
data_tmp = data.copy()

### Wykrywanie/usuwanie wartości odstających z wykorzystaniem średniej i odchylenie standardowego
- obcięło nam 20 wierszy

In [ ]:
factor = 3
upper_lim = data['Fare'].mean() + data['Fare'].std() * factor
lower_lim = data['Fare'].mean() - data['Fare'].std() * factor

data[(data['Fare'] < upper_lim) & (data['Fare'] > lower_lim)]

### Wykrywanie/usuwanie wartości odstających z wykorzystaniem percentyli
- usunęło ok. 200 wierszy

In [ ]:
upper_lim = data['Fare'].quantile(.90)
lower_lim = data['Fare'].quantile(.10)

data[(data['Fare'] < upper_lim) & (data['Fare'] > lower_lim)]

### Przycinanie (zastępowanie) wartości odstających

In [ ]:
data_tmp.loc[(data['Fare'] > upper_lim),'Fare'] = upper_lim
data_tmp.loc[(data['Fare'] < lower_lim),'Fare'] = lower_lim

In [ ]:
max(data.Fare), max(data_tmp.Fare)

In [ ]:
min(data.Fare), min(data_tmp.Fare)

- **Który sposób jest najlepszy?**
- **Kiedy będziemy usuwać a kiedy zastępować wartości odstające?**

## Binning
Co to jest i co nam to daje?
### Zmienne numeryczne

In [ ]:
# Przydatna funkcja w pandas do kubełkowania zmiennych numerycznych
bins = [0, 1, 5, 10, 25, 50, 600]
pd.cut(data['Fare'], bins)

### Zmienne kategoryczne

In [ ]:
geo=np.random.choice(("Poland",'Chile', 'France', 'Thailand'), 100)

In [ ]:
geo=pd.Series(geo)
geo

In [ ]:
#metoda z użyciem dict/defaultdict
dict_geo={'Poland': "Europe", "Chile":"South America", "France":"Europe"}
from collections import defaultdict
countries_list = [('Poland','Europe'), ('France','Europe'), ('Chile','South America')]

countries_dict = defaultdict(lambda:'Other')
for continent, country in countries_list:
     countries_dict[continent]=country

In [ ]:
geo.map(countries_dict)

## Transformations
Co to jest i do czego możemy to wykorzystać?

In [ ]:
plt.hist(data['Fare'])
plt.show()

In [ ]:
transformed_fare = (data['Fare']+1).transform(np.sqrt)
plt.hist(transformed_fare)
plt.show()

In [ ]:
transformed_fare = (data['Fare']+1).transform(np.log)
plt.hist(transformed_fare)
plt.show()

In [ ]:
plt.scatter(data['Fare'],data['Survived'])

In [ ]:
plt.scatter(transformed_fare,data['Survived'])

## Categorical encoding
Jakie są 2 główne typy zmiennych kategorycznych?

In [ ]:
data_arr = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = np.array(data_arr)

### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

# integer encode
le = LabelEncoder()
integer_encoded = le.fit_transform(values)
print(integer_encoded)
#invert
print(le.inverse_transform(integer_encoded))

### One Hot Encoding 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# one hot encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

# invert 
inverted = onehot_encoder.inverse_transform(onehot_encoded)
print(inverted.transpose())

### Target Encoding

In [ ]:
import category_encoders
te=category_encoders.target_encoder.TargetEncoder(data)

data_tmp = data.copy()
encoded=te.fit_transform(data['Cabin'],data['Survived'])
data_tmp['target_encoded'] = encoded

In [ ]:
data_tmp[['Cabin','Survived','target_encoded']]

In [ ]:
te=category_encoders.target_encoder.TargetEncoder(data)
encoded=te.fit_transform(data['Sex'],data['Survived'])
data_tmp['target_encoded'] = encoded

In [ ]:
data_tmp[['Sex','Survived','target_encoded']]

## Scalling
### Normalization
Normalizuje zmienne tak aby miały jednolitą normę - do czego można to wykorzystać?

In [ ]:
from sklearn import preprocessing
preprocessing.normalize(data[['Fare','Age']], norm = 'l1')

### Standarizing
Standaryzuje zmienną usuwając średnią i skalując przez wariancję:

$z = \frac{(x - mean)}{std}$

In [ ]:
names = ['Fare','Age']
scaler = preprocessing.StandardScaler()

scaled_df = scaler.fit_transform(data[['Fare','Age']])
pd.DataFrame(scaled_df, columns=names)